# <span style="color:#f37726">01 RKI Data Preperation</span>

**Ziele des Notebooks:**
- Attribute selektieren
- Datentypen anpassen und fehlende Werte ersetzen
- Landkreisdaten auslagern
- Gemeldete Fälle komprimieren
- Impffortschritt hinzufügen

In [1]:
# Benötigte Bibliotheken für dieses Notebook
import numpy as np
import pandas as pd

## Selektion der Attribute
Es soll entschieden werden, welche der 18 Attribute für das Vorhersagemodell relevant sind:

**FID** *≙ Index in der csv-Datei*
<br>Irrelevant. Primärschlüssel, jedoch ohne Bedeutung für die Vorhersage. Wird vom pandas DataFrame Index abgelöst.

**IdBundesland** *≙ Id des Bundeslands*	
Irrelevant. Name des Bundeslandes vorhanden.

**Bundesland** *≙ Name des Bundeslandes*
<br> Relevant. Ermöglicht geograhpische Modellierung.

**Landkreis** *≙ Name des Landkreises*
<br>Relevant. Ermöglicht geographische Modellierung

**Altersgruppe** *≙ Altersgruppe des Falles aus 6 Gruppen*
<br>Irrelevant. Fallzahlen sollen altersunabhänig vorhergesagt werden. Die Alterstruktur der Landkreise wird als demographischer Faktor im Clustering berücksichtigt.

**Geschlecht** *≙ Geschlecht des Falle*
<br>Irrelevant. Fallzahlen sollen unabhängig vom Geschlecht vorhergesagt werden.

**AnzahlFall** *≙ Anzahl der Fälle in der entsprechenden Gruppe*
<br>Relevant. Wird zu Fallzahlen akkumuliert.

**AnzahlTodesfall** *≙ Anzahl der Todesfälle in der entsprechenden Gruppe*
<br>Relevant. Wird (eventuell) zur Modellierung der infektionsgefährdeten Bevölkerung verwendet.

**Meldedatum** *≙ Datum, wann der Fall dem Gesundheitsamt bekannt wurde*
<br>Relevant. Ermöglicht zeitliche Modellierung.

**IdLandkreis** *≙ Id des Landkreises*
<br>Relevant. Erleichtert das Verbinden der Datensätze.

**Datenstand** *≙ Datum, wann der Datensatz zuletzt aktualisiert*
<br>Irrelevant. Der Datensatz wird täglich aktualisiert. Wir arbeiten mit der Version vom 13. April 2021.

**IdLandkreis** *≙ Id des Landkreises*
<br>Relevant. Erleichtert das Verbinden der Datensätze. Wird ebenfalls vom Statistischen Bundesamt verwendet.

**NeuerFall** *≙ Fall ist in Publikation des Vortages enthalten*
<br>Irrelevant. Meldefehler haben nur Einfluss auf dynamische Analysen (Echtzeit).

**NeuerTodesfall** *≙ Todesfall ist in Publikation des Vortages enthalten*
<br>Irrelevant. Meldefehler haben nur Einfluss auf dynamische Analysen (Echtzeit).

**Refdatum** *≙ Erkrankungsdatum*
<br>Irrelevant. Das Meldedatum wird zur zeitlichen Modellierung verwendet.

**NeuGenesen** *≙ Genesung ist in Publikation des Vortages enthalten*
<br>Irrelevant. Meldefehler haben nur Einfluss auf dynamische Analysen (Echtzeit).

**AnzahlGenesen** *≙ Anzahl der Genesenen in der entsprechenden Gruppe*
<br>Relevant. Wird (eventuell) zur Modellierung der infektionsgefährdeten Bevölkerung verwendet.

**IstErkrankungsbeginn** *≙ Ist Refdatum gleich Erkrankungsbeginn*
<br>Irrelevant. Das Meldedatum wird zur zeitlichen Modellierung verwendet.

**Altersgruppe2** *≙ Altersgruppe des Falles aus 5-Jahresgruppen*
<br>Irrelevant. Fallzahlen sollen altersunabhänig vorhergesagt werden. Die Alterstruktur der Landkreise wird als demographischer Faktor im Clustering berücksichtigt.

In [2]:
# Laden des Datensatzes als pandas DataFrame
data = pd.read_csv(r'data/RKI_COVID19.csv')
df = pd.DataFrame(data)

In [3]:
shape0 = df.shape[1]

# Entferne Spalten mit irrelevanten Attributen
df.drop("FID", axis=1, inplace=True)
df.drop("IdBundesland", axis=1, inplace=True)
df.drop("Altersgruppe", axis=1, inplace=True)
df.drop("Geschlecht", axis=1, inplace=True)
df.drop("Datenstand", axis=1, inplace=True)
df.drop("Refdatum", axis=1, inplace=True)
df.drop("IstErkrankungsbeginn", axis=1, inplace=True)
df.drop("Altersgruppe2", axis=1, inplace=True)
df.drop("NeuerFall", axis=1, inplace=True)
df.drop("NeuerTodesfall", axis=1, inplace=True)
df.drop("NeuGenesen", axis=1, inplace=True)

shape1 = df.shape[1]

print(f"Anzahl entfernter Attribute: {str(shape0-shape1)}")

Anzahl entfernter Attribute: 11


## Datenbereinigung
Fehlende Werte sollen entdeckt und ggf. mit sinnvollen Werten ersetzt werden:

In [4]:
# Erstelle Series mit NaN Zähler
nan_series = df.isna().sum()

# Erstelle Series mit Datentypen
types_series = df.dtypes

# Erstelle Series mit einzigartigen Werten
unique_series = df.nunique()

# Verschmelzen der Serien zu DataFrame
parameter_overview_df = pd.concat([unique_series, nan_series, types_series], axis=1)
parameter_overview_df.columns = ['distinct_values', 'NaN_count', 'dtype']
                                  
print(parameter_overview_df)
print("\n")
print("Form des DataFrame: " + str(df.shape))

                 distinct_values  NaN_count   dtype
Bundesland                    16          0  object
Landkreis                    412          0  object
AnzahlFall                   121          0   int64
AnzahlTodesfall               19          0   int64
Meldedatum                   431          0  object
IdLandkreis                  412          0   int64
AnzahlGenesen                121          0   int64


Form des DataFrame: (1652695, 7)


Die Datentypen der Attribute sind bereits kohärent mit den jeweils möglichen Ausprägungen. Lediglich das Meldedatum soll zu 'datetime64' umgewandelt werden. Boolsche Variablen existieren in ursprünglicher Form nicht. Die Attribute 'Landkreis', 'Bundesland' und 'Meldedatum' sind nominal skaliert. Die Attribute 'AnzahlFall', 'AnzahlTodesfall' und 'AnzahlGenesen' metrisch.

In [5]:
df['Meldedatum'] =  pd.to_datetime(df['Meldedatum'], format='%Y/%m/%d %H:%M:%S')
print(df['Meldedatum'].dtype)

datetime64[ns]


## Modifikation der Landkreisdaten

In [6]:
display(df.head())

,Bundesland,Landkreis,AnzahlFall,AnzahlTodesfall,Meldedatum,IdLandkreis,AnzahlGenesen
0,Schleswig-Holstein,SK Flensburg,1,0,2020-09-30,1001,1
1,Schleswig-Holstein,SK Flensburg,1,0,2020-10-29,1001,1
2,Schleswig-Holstein,SK Flensburg,1,0,2020-11-03,1001,1
3,Schleswig-Holstein,SK Flensburg,1,0,2020-11-20,1001,1
4,Schleswig-Holstein,SK Flensburg,1,0,2020-11-23,1001,1


Die Fallzahlen, die Todesfälle und die Genesungen sollen pro Tag und Landkreis akkumuliert werden. Die Auslagerung von Landkreisnamen und entsprechendem Bundesland in einen weiteren Dataframe erleichtert die Kompression:

In [7]:
landkreise_df = df[['Landkreis', 'Bundesland', 'IdLandkreis']]
landkreise_df = landkreise_df.groupby(['IdLandkreis']).last()
display(landkreise_df)

,Landkreis,Bundesland
IdLandkreis,,
1001,SK Flensburg,Schleswig-Holstein
1002,SK Kiel,Schleswig-Holstein
1003,SK Lübeck,Schleswig-Holstein
1004,SK Neumünster,Schleswig-Holstein
1051,LK Dithmarschen,Schleswig-Holstein
...,...,...
16073,LK Saalfeld-Rudolstadt,Thüringen
16074,LK Saale-Holzland-Kreis,Thüringen
16075,LK Saale-Orla-Kreis,Thüringen


Die Art des Kreises (Landkreis, kreisfreie Stadt) soll als boolsche Spalte ergänzt werden. Dazu wird eine selbstgeschriebene Funktion verwendet:

In [8]:
from rki_function0 import AddBoolColumn

landkreise_df['KreisArt']=landkreise_df.Landkreis.str[:2]
AddBoolColumn(landkreise_df, 'KreisArt', 'SK')
landkreise_df.drop("KreisArt", axis=1, inplace=True)
landkreise_df.rename(columns={"SK": "IstStadt"}, inplace=True)
landkreise_df['Landkreis'] = landkreise_df.Landkreis.str[3:]
display(landkreise_df)

,Landkreis,Bundesland,IstStadt
IdLandkreis,,,
1001,Flensburg,Schleswig-Holstein,True
1002,Kiel,Schleswig-Holstein,True
1003,Lübeck,Schleswig-Holstein,True
1004,Neumünster,Schleswig-Holstein,True
1051,Dithmarschen,Schleswig-Holstein,False
...,...,...,...
16073,Saalfeld-Rudolstadt,Thüringen,False
16074,Saale-Holzland-Kreis,Thüringen,False
16075,Saale-Orla-Kreis,Thüringen,False


Gemäß der Beobachtung im ersten Notebook sollen noch die Berliner Stadtbezirke zu einem Landkreis aggregiert werden:

In [9]:
berliner_ids_mask = landkreise_df.Landkreis.str.contains("Berlin")
berliner_ids = berliner_ids_mask.index[berliner_ids_mask]

# Entfernen der 12 Bezirke
landkreise_df.drop(berliner_ids, axis=0, inplace=True)

# Hinzufügen des Berliner Bezirks
landkreise_df.loc[11000] = ['Berlin', 'Berlin', True]
landkreise_df.sort_index(inplace=True)

Außerdem soll der DataFrame die Bevölkerungszahl des jeweiligen Landkreises enthalten. Das Statistische Bundesamt [veröffentlicht](https://www.destatis.de/DE/Themen/Laender-Regionen/Regionales/Gemeindeverzeichnis/Administrativ/04-kreise.html) den Bevölkerungstand jährlich als xlsx-Datei. Für das Einlesen haben wir eine csv-Datei erstellt:

In [10]:
data = pd.read_csv(r'data/landkreise_einwohner_fläche.csv', encoding='cp1252', delimiter=';')
landkreise_pop_df = pd.DataFrame(data)
landkreise_pop_df.set_index('IdLandkreis', drop=True, inplace=True)
display(landkreise_pop_df)
print(landkreise_pop_df.dtypes)

,Kreisart,Landkreis,Fläche_qkm,Bevölkerung,Bevölkerung_Männlich,Bevölkerung_Weiblich,Einwohner_pro_qkm
IdLandkreis,,,,,,,
1001,Kreisfreie Stadt,"Flensburg, Stadt",53.02,90164,44904,45260,1701
1002,Kreisfreie Stadt,"Kiel, Landeshauptstadt",118.65,246794,120198,126596,2080
1003,Kreisfreie Stadt,"Lübeck, Hansestadt",214.19,216530,104032,112498,1011
1004,Kreisfreie Stadt,"Neumünster, Stadt",71.66,80196,39723,40473,1119
1051,Kreis,Dithmarschen,1428.17,133193,65718,67475,93
...,...,...,...,...,...,...,...
16073,Landkreis,Saalfeld-Rudolstadt,1008.79,103199,50838,52361,102
16074,Landkreis,Saale-Holzland-Kreis,815.24,82950,41262,41688,102
16075,Landkreis,Saale-Orla-Kreis,1151.30,80312,39890,40422,70


Kreisart                 object
Landkreis                object
Fläche_qkm              float64
Bevölkerung               int64
Bevölkerung_Männlich      int64
Bevölkerung_Weiblich      int64
Einwohner_pro_qkm         int64
dtype: object


In [11]:
# Überflüssige Spalten entfernen
landkreise_pop_df.drop('Kreisart', axis=1, inplace=True)
landkreise_pop_df.drop('Landkreis', axis=1, inplace=True)
landkreise_pop_df.drop('Bevölkerung_Männlich', axis=1, inplace=True)
landkreise_pop_df.drop('Bevölkerung_Weiblich', axis=1, inplace=True)
landkreise_pop_df.drop('Einwohner_pro_qkm', axis=1, inplace=True)

# Landkreis DataFrame erweitern
landkreise_df = pd.concat([landkreise_df, landkreise_pop_df], axis=1)
display(landkreise_df)

,Landkreis,Bundesland,IstStadt,Fläche_qkm,Bevölkerung
IdLandkreis,,,,,
1001,Flensburg,Schleswig-Holstein,True,53.02,90164
1002,Kiel,Schleswig-Holstein,True,118.65,246794
1003,Lübeck,Schleswig-Holstein,True,214.19,216530
1004,Neumünster,Schleswig-Holstein,True,71.66,80196
1051,Dithmarschen,Schleswig-Holstein,False,1428.17,133193
...,...,...,...,...,...
16073,Saalfeld-Rudolstadt,Thüringen,False,1008.79,103199
16074,Saale-Holzland-Kreis,Thüringen,False,815.24,82950
16075,Saale-Orla-Kreis,Thüringen,False,1151.30,80312


## Kompression der gemeldeten Fälle

Die Fallzahlen sollen für jeden Landkreis als Tageswert vorliegen. Für die Umsetzung bietet sich [pandas gropby-Funktion](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html) in Kombination mit einem Multiindex an:

In [12]:
df.drop("Bundesland", axis=1, inplace=True)
df.drop("Landkreis", axis=1, inplace=True)

df = df.groupby(['IdLandkreis', 'Meldedatum']).sum() # Multiindex

display(df)

AnzahlFall  AnzahlTodesfall  AnzahlGenesen
IdLandkreis Meldedatum                                            
1001        2020-03-14           4                0              4
            2020-03-18           2                0              2
            2020-03-19           4                0              4
            2020-03-20           2                0              2
            2020-03-21           1                0              1
...                            ...              ...            ...
16077       2021-04-08          50                0              0
            2021-04-09          37                0              0
            2021-04-10           7                0              0
            2021-04-11          13                0              0
            2021-04-12          23                0              0

[122522 rows x 3 columns]

Der zeitliche Horizont der Meldedaten soll angepasst werden. Beginnend ab März 2020 bis April 2021 werden dann auch Tage ohne gemeldete Fälle im Datensatz ergänzt:

In [13]:
start_time = pd.to_datetime('2020-03-01')
end_time = pd.to_datetime('2021-03-31')
case_dates = pd.date_range(start_time, end_time, freq='D')


df = df.loc[(slice(None), case_dates), :]
df = df.reindex(pd.MultiIndex.from_product([df.index.levels[0], case_dates], names=['IdLandkreis', 'Meldedatum']), fill_value=0)
display(df)

AnzahlFall  AnzahlTodesfall  AnzahlGenesen
IdLandkreis Meldedatum                                            
1001        2020-03-01           0                0              0
            2020-03-02           0                0              0
            2020-03-03           0                0              0
            2020-03-04           0                0              0
            2020-03-05           0                0              0
...                            ...              ...            ...
16077       2021-03-27          27                0              0
            2021-03-28           9                0              0
            2021-03-29          29                0              0
            2021-03-30          38                0              0
            2021-03-31          79                0              3

[163152 rows x 3 columns]

Abschließend sollen auch in diesem DataFrame die Berliner Bezirke zu einem Landkreis aggregiert werden. Dafür wird erneut eine selbstgeschriebene Funktion verwendet:

In [14]:
from rki_function1 import CountyCasesAggregator


df = CountyCasesAggregator(df, berliner_ids, 11000)

print("Berlin hinzugefügt?: " + str(11000 in df.index))
print("\n")
print("Berliner Bezirke entfernt?:")
for id in berliner_ids:
    print(id not in df.index.get_level_values(0).tolist())

C:\Users\Niklas\anaconda3\lib\site-packages\pandas\core\generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


Berlin hinzugefügt?: True


Berliner Bezirke entfernt?:
True
True
True
True
True
True
True
True
True
True
True
True


In einem weiteren DataFrame wird zu den Kalenderdaten noch der Wochentag und Monat gespeichert:

In [15]:
case_dates_df = pd.DataFrame(case_dates, columns=['Meldedatum'])
case_dates_df['Monat']=case_dates_df['Meldedatum'].dt.month
case_dates_df['Wochentag'] = case_dates_df['Meldedatum'].dt.dayofweek
case_dates_df['Jahr'] = case_dates_df['Meldedatum'].dt.year
case_dates_df.set_index('Meldedatum', drop=True, inplace=True)
display(case_dates_df)

,Monat,Wochentag,Jahr
Meldedatum,,,
2020-03-01,3,6,2020
2020-03-02,3,0,2020
2020-03-03,3,1,2020
2020-03-04,3,2,2020
2020-03-05,3,3,2020
...,...,...,...
2021-03-27,3,5,2021
2021-03-28,3,6,2021
2021-03-29,3,0,2021


## Impffortschritt hinzufügen

Für die Modellierung der ansteckungsgefährdenden Bevölkerung soll der Impffortschritt berücksichtigt werden. Die Daten dazu stellt das Robert Koch-Institut zusammen mit Visualisierungen in einem [Dashboard](https://impfdashboard.de/) bereit:

In [16]:
data = pd.read_csv(r'data/germany_vaccinations_timeseries_v2.tsv', sep='\t')
vaccines_df = pd.DataFrame(data)
display(vaccines_df)

,date,dosen_kumulativ,dosen_differenz_zum_vortag,dosen_erst_differenz_zum_vortag,dosen_zweit_differenz_zum_vortag,dosen_biontech_kumulativ,dosen_moderna_kumulativ,dosen_astrazeneca_kumulativ,personen_erst_kumulativ,personen_voll_kumulativ,...,indikation_pflegeheim_voll,dosen_dim_kumulativ,dosen_kbv_kumulativ,dosen_johnson_kumulativ,dosen_biontech_erst_kumulativ,dosen_biontech_zweit_kumulativ,dosen_moderna_erst_kumulativ,dosen_moderna_zweit_kumulativ,dosen_astrazeneca_erst_kumulativ,dosen_astrazeneca_zweit_kumulativ
0,2020-12-27,23999,23999,23986,13,23998,1,0,23986,13,...,68,23999,0,0,23985,13,1,0,0,0
1,2020-12-28,42484,18485,18448,37,42483,1,0,42434,50,...,68,42484,0,0,42433,50,1,0,0,0
2,2020-12-29,93219,50735,50010,725,93218,1,0,92444,775,...,409,93219,0,0,92443,775,1,0,0,0
3,2020-12-30,155891,62672,62552,120,155890,1,0,154996,895,...,412,155891,0,0,154995,895,1,0,0,0
4,2020-12-31,205910,50019,49870,149,205909,1,0,204866,1044,...,489,205910,0,0,204865,1044,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,2021-05-07,34530528,864390,642326,222064,25553504,2196351,6760476,26925761,7624964,...,760340,26796646,7733882,20197,18627278,6926226,1669777,526574,6608509,151967
132,2021-05-08,34945900,415372,266787,148585,25816251,2265475,6843110,27193415,7773549,...,760340,27151527,7794373,21064,18780008,7036243,1720062,545413,6672281,170829
133,2021-05-09,35222512,276612,177042,99570,26007599,2317416,6875393,27371497,7873119,...,760340,27417104,7805408,22104,18893512,7114087,1761583,555833,6694298,181095
134,2021-05-10,35789319,566807,384810,181997,26366932,2393300,7003765,27759525,8055116,...,760340,27795013,7994306,25322,19117478,7249454,1818393,574907,6798332,205433


Wichtig für die Modellierung ist lediglich die bundesweite Quote an Erst- und Vollimpfungen. Zwischen den Impfstoffen soll nicht unterschieden werden. Auch die Priorisierung wird vernachlässigt:

In [17]:
# Entferne Spalten mit irrelevanten Attributen
vaccines_df = vaccines_df[['date', 'impf_quote_erst', 'impf_quote_voll']]
vaccines_df.rename(columns={"date": "Meldedatum"}, inplace=True)
vaccines_df['Meldedatum'] =  pd.to_datetime(vaccines_df['Meldedatum'], format='%Y/%m/%d %H:%M:%S')
vaccines_df.set_index('Meldedatum', drop=True, inplace=True)
display(vaccines_df)

,impf_quote_erst,impf_quote_voll
Meldedatum,,
2020-12-27,0.000,0.000
2020-12-28,0.001,0.000
2020-12-29,0.001,0.000
2020-12-30,0.002,0.000
2020-12-31,0.002,0.000
...,...,...
2021-05-07,0.324,0.092
2021-05-08,0.327,0.093
2021-05-09,0.329,0.095


Die Exploration der komprimierten Fallzahlen ist Thema des nächsten Notebooks. Dazu sollen die DataFrames als pickle bzw. csv-Datein exportiert werden:

In [18]:
df.to_pickle(r'data/fallzahlen_akkumuliert.pkl')
df.to_csv(r'data/fallzahlen_akkumuliert.csv', sep=';', encoding='utf-8', index=True)

landkreise_df.to_pickle(r'data/landkreise_bundesland_id.pkl')
landkreise_df.to_csv(r'data/landkreise_bundesland_id.csv', sep=';', encoding='utf-8', index=True)

case_dates_df.to_pickle(r'data/case_dates.pkl')
case_dates_df.to_csv(r'data/case_dates.csv', sep=';', encoding='utf-8', index=True)

vaccines_df.to_pickle(r'data/vaccine_count.pkl')
vaccines_df.to_csv(r'data/vaccine_count.csv', sep=';', encoding='utf-8', index=True)